In [ ]:
# Cell 1: Imports and environment checks
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
print("TensorFlow version:", tf.__version__)
%matplotlib inline

In [ ]:
# Cell 2: Load MNIST (we only need it for completeness; inference uses noise)
(train_ds, test_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_superv
def normalize_img(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.expand_dims(image, -1)  # ensure shape (28,28,1)
    return image, label

train_ds = train_ds.map(normalize_img).batch(128)
print("MNIST loaded. Example batch shape:", next(iter(train_ds))[0].shape

In [ ]:
# Cell 3: Diffusion schedule
T = 300  # number of timesteps (you can reduce to 100 for faster runs)
beta = np.linspace(1e-4, 0.02, T, dtype=np.float32)
alpha = 1.0 - beta
alpha_bar = np.cumprod(alpha)
print("Schedule ready. T =", T)

In [ ]:
# Cell 4: Simple UNet-like model (academic)
def build_unet():
    inputs = tf.keras.layers.Input(shape=(28, 28, 1))
    x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')(inputs)
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.UpSampling2D()(x)
    x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    outputs = tf.keras.layers.Conv2D(1, 3, padding='same')(x)
    return tf.keras.Model(inputs, outputs, name="Simple_UNet")

model = build_unet()
model.summary()

In [ ]:
# Cell 5: Reverse (denoising) step — corrected version (no time concat)
def denoise_step(x_t, t):
    """
    Simulates a reverse diffusion step using the model's predicted noise.
    Note: We are NOT concatenating a time-channel here to keep shapes consistent
    with the Simple_UNet defined above.
    """
    pred_noise = model(x_t, training=False)  # model predicts noise residual
    alpha_t = alpha[t]
    alpha_bar_t = alpha_bar[t]
    beta_t = beta[t]
    # Reverse diffusion update (academic / simplified)
    x_prev = (1.0 / np.sqrt(alpha_t)) * (x_t - ((1.0 - alpha_t) / np.sqrt(1.0 - alpha
    if t > 0:
        noise = tf.random.normal(x_t.shape)
        x_prev = x_prev + np.sqrt(beta_t) * noise
    return x_pre

In [ ]:
# Cell 6: Generate images from pure noise
num_images = 5
x_t = tf.random.normal((num_images, 28, 28, 1))  # start from pure Gaussian noise

# Steps we will capture for visualization
steps_to_show = [0, 50, 100, 150, 200, 250, T-1]
generated_images = []

print("Running reverse diffusion...")

for t in tqdm(reversed(range(T)), desc="Denoising"):
    x_t = denoise_step(x_t, t)
    if t in steps_to_show:
        generated_images.append(x_t.numpy())

print("Done denoising. Captured", len(generated_images), "snapshots.")

In [ ]:
# Cell 7: Visualization
fig, axes = plt.subplots(num_images, len(steps_to_show), figsize=(12, 6))
for i in range(num_images):
    for j, img in enumerate(generated_images):
        axes[i, j].imshow(np.clip(img[i, :, :, 0], -1, 1), cmap='gray')
        axes[i, j].axis('off')
        if i == 0:
            axes[i, j].set_title(f"Step {steps_to_show[j]}")

plt.suptitle("MNIST Image Generation using Diffusion Process (academic simulation)")
plt.show()

In [ ]:
# Cell 8: Save final images to disk
import os
os.makedirs('ddpm_outputs', exist_ok=True)
final_imgs = generated_images[-1]  # last snapshot
for i in range(final_imgs.shape[0]):
    plt.imsave(f'ddpm_outputs/generated_{i}.png', np.clip(final_imgs[i, :, :, 0], -1), 1)
print("Saved images to /content/ddpm_outputs/")